In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model = "llama3.2:3b",
    temperature = 0,
    base_url='http://localhost:11434'
)

Usando apenas o LLM

In [3]:
response = llm.invoke("Qual capitulo da lgpd trata do tratamento de dados pessoais pelo poder publico?")
print(response.content)


Olá!

A Lei Geral de Proteção de Dados (LGPD) é uma lei brasileira que regula a proteção de dados pessoais e a sua utilização por parte de entidades públicas.

O capítulo específico da LGPD que trata do tratamento de dados pessoais pelo poder público é o Capítulo VIII, que é composto pelos artigos 43 a 55.

Aqui estão alguns pontos importantes relacionados ao tratamento de dados pessoais pelo poder público, conforme estabelecido na LGPD:

*   Art. 43: O Poder Público deve garantir a proteção dos dados pessoais e a sua utilização apenas para fins previstos em lei.
*   Art. 44: A coleta, armazenamento e tratamento de dados pessoais pelo Poder Público devem ser realizados por meio de instrumentos legais ou regulamentares que garantam a proteção dos dados.
*   Art. 45: O Poder Público deve informar os cidadãos sobre as razões da coleta, armazenamento e tratamento de seus dados pessoais.
*   Art. 46: A coleta, armazenamento e tratamento de dados pessoais pelo Poder Público devem ser realiza

Obtemos a resposta errada

In [4]:
import pdfplumber
from pytesseract import image_to_string

def extract_text_with_ocr(pdf_path):
    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:

            if page.extract_text():
                extracted_text.append(page.extract_text())
            else:
                page_image = page.to_image(resolution=300)
                image = page_image.original  
                text = image_to_string(image)
                extracted_text.append(text)
    return extracted_text




extração dos dados do pdf

In [5]:
pdf_path = "data/lgpd.pdf"
text = extract_text_with_ocr(pdf_path)

print(text[:50])




['DIARIO OFICIAL DA UNIAO\n\nPublicado em: 15/08/2018 | Edic&o: 157 | Segao: 1 | Pagina: 59\nOrgao: Atos do Poder Legislativo\n\nLEI NO 13.709, DE 14 DE AGOSTO DE 2018\n\nDisp6e sobre a protegao de dados pessoais e altera a Lei n°\n12.965, de 23 de abril de 2014 (Marco Civil da Internet).\n\nOPRESIDENTEDAREPUBLICA\nFago saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:\nCAPITULO |\nDISPOSICOES PRELIMINARES\n\nArt. 1° Esta Lei dispoe sobre o tratamento de dados pessoais, inclusive nos meios digitais, por\npessoa natural ou por pessoa juridica de direito publico ou privado, com o objetivo de proteger os direitos\nfundamentais de liberdade e de privacidade e o livre desenvolvimento da personalidade da pessoa\nnatural.\n\nArt. 2° A disciplina da protecao de dados pessoais tem como fundamentos:\n| - o respeito a privacidade:\n\nIl - a autodeterminacao informativa;\n\nIll - a liberdade de expressao, de informacao, de comunicagao e de opiniao;\nIV - a inviolabilidade da int

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document, HumanMessage
from langchain.vectorstores import FAISS



def create_documents(texts):
    if isinstance(texts, list):
        documents = [Document(page_content=text) for text in texts]
    else:
        documents = [Document(page_content=texts)]
    return documents

def create_faiss_vectorstore(documents):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore

def generate_response_with_ollama(vectorstore, query):
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.invoke(query)
    context = "\n".join([doc.page_content for doc in retrieved_docs])
    
    print("Contexto Recuperado:")
    print(context[:200])
    
    prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
    response = llm.invoke([HumanMessage(content=prompt)])
    return response.content.strip()

def rag_pipeline_with_ollama(texts, query):
    documents = create_documents(texts)
    vectorstore = create_faiss_vectorstore(documents)
    response = generate_response_with_ollama(vectorstore, query)
    return response




In [7]:
query = "Qual capítulo da LGPD trata do tratamento de dados pessoais pelo poder público?"
response = rag_pipeline_with_ollama(text, query)

# Exibir resposta gerada
print("\nResposta Gerada:", response)

/home/airex/anaconda3/envs/rag/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Contexto Recuperado:
DIARIO OFICIAL DA UNIAO

Publicado em: 15/08/2018 | Edic&o: 157 | Segao: 1 | Pagina: 59
Orgao: Atos do Poder Legislativo

LEI NO 13.709, DE 14 DE AGOSTO DE 2018

Disp6e sobre a protegao de dados pesso

Resposta Gerada: O capítulo que trata do tratamento de dados pessoais pelo poder público é o Capítulo IV, "Tratamento de Dados Pessoais por Poder Público".


multiplos_documentos

In [8]:
# import os
# import pdfplumber
# from pytesseract import image_to_string
# from langchain.vectorstores import FAISS
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.schema import Document
# from langchain_ollama import ChatOllama


# def extract_text_with_ocr(pdf_path):

#     extracted_text = []
#     with pdfplumber.open(pdf_path) as pdf:
#         for page in pdf.pages:
#             if page.extract_text():
#                 extracted_text.append(page.extract_text())
#             else:
#                 page_image = page.to_image(resolution=300)
#                 image = page_image.original
#                 text = image_to_string(image, lang="por")
#                 extracted_text.append(text)
#     return "\n".join(extracted_text) 


# def load_pdf_documents(pdf_paths):
#     all_documents = []
#     for path in pdf_paths:
#         print(f"Carregando: {path}")
#         try:
#             # Extract and consolidate text from the PDF
#             text = extract_text_with_ocr(path)
#             filename = os.path.basename(path)
#             document = Document(
#                 page_content=text,
#                 metadata={"source": filename}  # Metadata includes the file name
#             )
#             all_documents.append(document)
#             print(f"Documento consolidado para {filename}")
#         except Exception as e:
#             print(f"Erro ao carregar {path}: {e}")

#     if not all_documents:
#         raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
#     print(f"Total de documentos carregados: {len(all_documents)}")
#     return all_documents


# def create_vectorstore(documents):
#     embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     vectorstore = FAISS.from_documents(documents, embeddings)
#     return vectorstore


# def generate_response(query, vectorstore):
#     docs = vectorstore.similarity_search(query, k=5)
#     context = "\n\n".join([
#         f"Source: {doc.metadata.get('source', 'Unknown')}\nContent: {doc.page_content}" for doc in docs
#     ])
    
#     llm = ChatOllama(
#         model="llama3.2:3b",
#         temperature=0,
#         base_url="http://localhost:11434"
#     )

#     prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
#     response = llm.invoke([prompt])
    
#     output = {
#         "question": query,
#         "context": context,
#         "response": response.content.strip(),
#         "sources": [doc.metadata.get("source", "Unknown") for doc in docs]
#     }
#     return output


# def rag_pipeline_for_pdfs(pdf_paths, query):

#     documents = load_pdf_documents(pdf_paths)
#     vectorstore = create_vectorstore(documents)
#     response = generate_response(query, vectorstore)
#     return response


# # Example Execution
# pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
# query = "Quais são os princípios da LGPD?"
# response = rag_pipeline_for_pdfs(pdf_files, query)

# # Print detailed response
# print("\nResposta Gerada:")
# print(response["response"])
# print("Pergunta:", response["question"])
# print("\nContexto:")
# print(response["context"][:200])
# print("\nFontes:")
# for source in set(response["sources"]):
#     print("-", source)


In [9]:
import os
import pdfplumber
from pytesseract import image_to_string
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_ollama import ChatOllama


In [10]:
def extract_text_with_ocr(pdf_path):

    extracted_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            if page.extract_text():
                extracted_text.append(page.extract_text())
            else:
                page_image = page.to_image(resolution=300)
                image = page_image.original
                text = image_to_string(image, lang="por")
                extracted_text.append(text)
    return "\n".join(extracted_text)


In [11]:
def load_pdf_documents(pdf_paths):

    all_documents = []
    for path in pdf_paths:
        print(f"Carregando: {path}")
        try:
            # Extract and consolidate text from the PDF
            text = extract_text_with_ocr(path)
            filename = os.path.basename(path)
            document = Document(
                page_content=text,
                metadata={"source": filename}  # Metadata includes the file name
            )
            all_documents.append(document)
            print(f"Documento consolidado para {filename}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {e}")

    if not all_documents:
        raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
    print(f"Total de documentos carregados: {len(all_documents)}")
    return all_documents


In [12]:
def create_vectorstore(documents):

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(documents, embeddings)
    return vectorstore


In [13]:
def generate_response(query, vectorstore):

    docs = vectorstore.similarity_search(query, k=5)
    context = "\n\n".join([
        f"Source: {doc.metadata.get('source', 'Unknown')}\nContent: {doc.page_content}" for doc in docs
    ])
    
    llm = ChatOllama(
        model="llama3.2:3b",
        temperature=0,
        base_url="http://localhost:11434"
    )

    prompt = f"Contexto: {context}\n\nPergunta: {query}\nResposta:"
    response = llm.invoke([prompt])
    
    output = {
        "question": query,
        "context": context[:500],
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Unknown") for doc in docs]
    }
    return output


In [14]:
def prepare_vectorstore(pdf_paths):

    documents = load_pdf_documents(pdf_paths)
    vectorstore = create_vectorstore(documents)
    print("Vectorstore criado com sucesso.")
    return vectorstore


In [15]:
def ask_question(query, vectorstore):

    response = generate_response(query, vectorstore)
    return response


In [16]:
pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore = prepare_vectorstore(pdf_files)

Carregando: data/lgpd.pdf
Documento consolidado para lgpd.pdf
Carregando: data/codigo_civil.pdf
Documento consolidado para codigo_civil.pdf
Total de documentos carregados: 2


/tmp/ipykernel_2234/2212122749.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Vectorstore criado com sucesso.


In [17]:
# Fazer uma pergunta
query1 = "Quais são os princípios da LGPD?"
response1 = ask_question(query1, vectorstore)

# Exibir a resposta
print("\nResposta Gerada:")
print(response1["response"])
print("Pergunta:", response1["question"])
print("Contexto:", response1["context"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Fazer outra pergunta sem recarregar os documentos
query2 = "Qual capitulo trata da sociedade anonima no código civil?"
response2 = ask_question(query2, vectorstore)

# Exibir a nova resposta
print("\nResposta Gerada:")
print(response2["response"])
print("Contexto:", response2["context"])

print("Pergunta:", response2["question"])

print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)



Resposta Gerada:
Os princípios da Lei Geral de Proteção de Dados (LGPD) incluem a proteção dos dados pessoais, a transparência e a responsabilidade, a consentimento informado, a segurança e a confidencialidade, a privacidade e a proteção contra o uso indevido.
Pergunta: Quais são os princípios da LGPD?
Contexto: Source: lgpd.pdf
Content: DIÁRIO OFICIAL DA UNIÃO

Publicado em: 15/08/2018 | Edição: 157 | Seção: 1 | Página: 59
Órgão: Atos do Poder Legislativo

LEI NO 13.709, DE 14 DE AGOSTO DE 2018

Dispõe sobre a proteção de dados pessoais e altera a Lei nº
12.965, de 23 de abril de 2014 (Marco Civil da Internet).

OPRESIDENTEDAREPÚBLICA
Faço saber que o Congresso Nacional decreta e eu sanciono a seguinte Lei:
CAPÍTULO |
DISPOSIÇÕES PRELIMINARES

Art. 1º Esta Lei dispõe sobre o tratamento de dados pessoai

Fontes:
- lgpd.pdf
- codigo_civil.pdf

Resposta Gerada:
O Capítulo III do Código Civil Brasileiro trata da Sociedade Anônima.
Contexto: Source: lgpd.pdf
Content: DIÁRIO OFICIAL DA UNI

usar chunks

In [18]:
def split_text_into_chunks(text, chunk_size=1000, overlap=200):

    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap if end - overlap > start else end
    return chunks


In [19]:
def load_pdf_documents_with_chunks(pdf_paths, chunk_size=1000, overlap=200):

    all_documents = []
    for path in pdf_paths:
        print(f"Carregando: {path}")
        try:
            text = extract_text_with_ocr(path)
            filename = os.path.basename(path)
            chunks = split_text_into_chunks(text, chunk_size, overlap)
            for i, chunk in enumerate(chunks):
                document = Document(
                    page_content=chunk,
                    metadata={"source": filename, "chunk_index": i}
                )
                all_documents.append(document)
            print(f"{len(chunks)} chunks criados para {filename}")
        except Exception as e:
            print(f"Erro ao carregar {path}: {e}")

    if not all_documents:
        raise ValueError("Nenhum documento foi carregado. Verifique os caminhos dos PDFs.")
    
    print(f"Total de chunks carregados: {len(all_documents)}")
    return all_documents


In [20]:
def prepare_vectorstore_with_chunks(pdf_paths, chunk_size=1000, overlap=200):

    documents = load_pdf_documents_with_chunks(pdf_paths, chunk_size, overlap)
    vectorstore = create_vectorstore(documents)
    print("Vectorstore criado com chunks.")
    return vectorstore


In [21]:
# Carregar documentos, dividir em chunks e criar vetorstore
pdf_files = ["data/lgpd.pdf", "data/codigo_civil.pdf"]
vectorstore_with_chunks = prepare_vectorstore_with_chunks(pdf_files, chunk_size=1000, overlap=200)


Carregando: data/lgpd.pdf
72 chunks criados para lgpd.pdf
Carregando: data/codigo_civil.pdf
1666 chunks criados para codigo_civil.pdf
Total de chunks carregados: 1738
Vectorstore criado com chunks.


In [22]:
# Fazer uma pergunta
query1 = "definicao lei geral de protecao de dados"
response1 = ask_question(query1, vectorstore_with_chunks)

# Exibir a resposta
print("\nResposta Gerada:")
print(response1["response"])
print("Pergunta:", response1["question"])


print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Fazer outra pergunta sem recarregar os documentos
query2 = "capitulo das perdas e danos?"
response2 = ask_question(query2, vectorstore_with_chunks)

# Exibir a nova resposta
print("\nResposta Gerada:")
print(response2["response"])


print("Pergunta:", response2["question"])

print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)



Resposta Gerada:
A Lei Geral de Proteção de Dados Pessoais (LGPD) define a proteção de dados pessoais como um direito fundamental que visa garantir a privacidade e a segurança dos indivíduos em relação à coleta, armazenamento e tratamento de suas informações pessoais. A LGPD estabelece regras e princípios para proteger os dados pessoais, incluindo:

1. Princípio da transparência: é obrigatório informar o titular dos dados sobre a coleta, uso e compartilhamento de seus dados.
2. Princípio da consentimento: é necessário obter o consentimento do titular para coletar, armazenar e tratar seus dados pessoais.
3. Princípio da segurança: os dados pessoais devem ser protegidos contra acessos não autorizados, perda, dano ou destruição.
4. Princípio da confidencialidade: os dados pessoais devem ser mantidos confidenciais e não compartilhados sem consentimento do titular.
5. Princípio da responsabilidade: os controladores e operadores de dados são responsáveis por garantir a proteção dos dados pe

In [44]:
def summarize_history_with_previous(history):
    """
    Inclua todo o histórico, destacando explicitamente a interação mais recente.
    """
    if not history:
        return "Histórico vazio."

    # Última interação (pergunta e resposta anterior)
    last_interaction = f"Pergunta Anterior: {history[-1][0]}\nResposta Anterior: {history[-1][1]}"
    
    # Outras interações (exceto a última)
    previous_interactions = "\n".join([
        f"Pergunta: {q}\nResposta: {r}" for q, r in history[:-1]
    ])
    
    # Combinar tudo
    return f"{previous_interactions}\n\n{last_interaction}" if previous_interactions else last_interaction


def format_prompt_with_history(query, context, history):
    """
    Gera o prompt incluindo o histórico completo de interações e o contexto.
    Integra explicitamente a última pergunta e resposta no contexto.
    """
    # Incluir o histórico completo
    full_history = summarize_history_with_previous(history)
    
    # Destacar a última interação diretamente no contexto
    if history:
        last_question, last_answer = history[-1]
        last_interaction_context = f"Baseando-se na última interação:\nPergunta: {last_question}\nResposta: {last_answer}\n"
    else:
        last_interaction_context = ""

    # Construir o prompt
    prompt = f"""
    Você é um assistente jurídico especializado no Código Civil brasileiro.

    Última Interação:
    {last_interaction_context}

    Histórico de Conversa:
    {full_history}

    Trechos relevantes dos documentos:
    {context}

    Nova Pergunta:
    {query}

    Resposta:
    """
    return prompt


def generate_response_with_history(query, vectorstore, history):
    """
    Gera resposta considerando o histórico completo, com destaque para a pergunta anterior, e o contexto relevante.
    """
    # Buscar os documentos mais relevantes no Vectorstore
    docs = vectorstore.similarity_search(query, k=5)
    
    # Limitar o contexto ao mais relevante
    context = "\n\n".join([
        f"Fonte: {doc.metadata.get('source', 'Desconhecido')}\nTrecho: {doc.page_content[:300]}" for doc in docs
    ])
    
    # Configurar o LLM
    llm = ChatOllama(
        model="llama3.2:3b",
        temperature=0,
        base_url="http://localhost:11434"
    )
    
    # Gerar o prompt com histórico e contexto
    prompt = format_prompt_with_history(query, context, history)
    
    # Obter a resposta do modelo
    response = llm.invoke([prompt])
    
    # Adicionar a interação ao histórico ANTES de retornar
    history.append((query, response.content.strip()))
    
    # Formatar e retornar o resultado
    output = {
        "question": query,
        "response": response.content.strip(),
        "sources": [doc.metadata.get("source", "Desconhecido") for doc in docs],
        "context_used": context  # Adicionado para debugging
    }
    return output


def ask_question_with_history(query, vectorstore, history):
    """
    Wrapper para gerar respostas com histórico.
    """
    return generate_response_with_history(query, vectorstore, history)


# Exemplo de Uso com Histórico Corrigido
conversation_history = []

# Pergunta 1
response1 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response1["response"])
print("\nFontes:")
for source in set(response1["sources"]):
    print("-", source)

# Pergunta 2 (Nova pergunta com base na resposta anterior)
response2 = ask_question_with_history(
    '''Os capítulos do Livro II do Código Civil são Livro II – Dos Bens
Título Único – Das Diferentes Classes de Bens

Capítulo I – Dos Bens Considerados em Si Mesmos
Seção I – Dos Bens Imóveis
Seção II – Dos Bens Móveis
Seção III – Dos Bens Fungíveis e Consumíveis
Seção IV – Dos Bens Divisíveis
Seção V – Dos Bens Singulares e Coletivos
Capítulo II – Dos Bens Reciprocamente Considerados
Capítulo III – Dos Bens Públicos''', 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response2["response"])
print("\nFontes:")
for source in set(response2["sources"]):
    print("-", source)

# Pergunta 3 (Outra pergunta baseada no fluxo de conversa)
response3 = ask_question_with_history(
    "Quais são os capítulos e seções do Livro II do Código Civil?", 
    vectorstore_with_chunks, 
    conversation_history
)

print("\nResposta Gerada:")
print(response3["response"])
print("\nFontes:")
for source in set(response3["sources"]):
    print("-", source)



Resposta Gerada:
Olá! Estou aqui para ajudar.

O Livro II do Código Civil brasileiro é dividido em várias seções, que abordam diferentes aspectos do direito privado. Aqui estão os capítulos e seções principais:

**Capítulo I: Do Testamento**

* Seção 1: Da vontade testamental
* Seção 2: Dos testamentos autênticos

**Capítulo II: Da Dote**

* Seção 1: Da dote em geral
* Seção 2: Da dote de casamento
* Seção 3: Da dote de filhos

**Capítulo III: Do Matrimônio**

* Seção 1: Da capacidade para o matrimônio
* Seção 2: Das condições do matrimônio
* Seção 3: Dos requisitos do casamento
* Seção 4: Da celebração e da homologação do casamento

**Capítulo IV: Da Adoção**

* Seção 1: Da adoção em geral
* Seção 2: Do ato de adoção
* Seção 3: Dos requisitos para a adoção
* Seção 4: Da adoção especial (inclusa a adoção por substituição)

**Capítulo V: Da Fideicomissão**

* Seção 1: Da fideicomissão em geral
* Seção 2: Do ato de fideicomissão

Essas são as seções principais do Livro II do Código Civi

usar agent

In [23]:
from langchain.agents import Tool
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA



retrieval_qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore_with_chunks.as_retriever())

tools = [
    Tool(
        name="Document Search",
        func=retrieval_qa.run,
        description="Use para buscar informações relevantes nos documentos. Perguntas devem ser específicas."
    )
]


In [24]:
from langchain.agents import initialize_agent, AgentType

# Inicializar o Agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True ,
    handle_parsing_errors=True,
    max_iterations=1

)


/tmp/ipykernel_2234/1202134613.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(


In [29]:
# Consultar o Agent
query1 = "definicao lei geral de protecao de dados"
query2 = "capitulo das perdas e danos?"

response1 = agent.run(query1)
response2 = agent.run(query2)

# Exibir as respostas
print("Resposta 1:", response1)
print("Resposta 2:", response2)




> Entering new AgentExecutor chain...
Aqui está a resposta:

Question: definicao lei geral de protecao de dados

Thought: Preciso encontrar informações sobre a Lei Geral de Proteção de Dados (LGPD) no Brasil.

Action: Document Search
Action Input: "Lei Geral de Proteção de Dados LGPD"
Observation: A Lei Geral de Proteção de Dados (LGPD) é uma lei brasileira que regula a proteção de dados pessoais. Ela foi publicada em 14 de agosto de 2018 e entrou em vigor em 15 de setembro de 2018.

A LGPD tem como objetivo proteger os direitos fundamentais da liberdade e da privacidade, bem como o livre desenvolvimento da personalidade da pessoa natural. Ela estabelece regras para a coleta, armazenamento, tratamento e compartilhamento de dados pessoais.

Algumas das principais disposições da LGPD incluem:

* A necessidade de obter consentimento informado do titular dos dados antes de coletar e processá-los;
* A obrigação de proteger os dados pessoais contra violações e danos;
* A criação de um órgã

outro metodo de rag

In [26]:
from langchain.chains import RetrievalQA

# Configurar chain de recuperação e geração
retrieval_qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True  # Retorna também os documentos utilizados
)


In [ ]:
query = "codigo civil capitulo dos contratos em geral?"
response = retrieval_qa.invoke({"query": query})

print("\nResposta Gerada:")
print(response["result"])

print("\nDocumentos Utilizados:")
for doc in response["source_documents"]:
    print(f"- Fonte: {doc.metadata['source']}")
    print(f"Trecho: {doc.page_content[:200]}...\n")  # Mostra apenas os primeiros 200 caracteres



Resposta Gerada:
O Capítulo II do Código Civil Brasileiro (CC) aborda os contratos em geral. Aqui está uma visão geral dos principais pontos:

**Artigos relevantes:**

* Artigo 129: Definição de contrato
* Artigo 130: Tipos de contratos
* Artigo 131: Condições necessárias para a formação do contrato
* Artigo 132: Condições da vontade para o contrato
* Artigo 133: Obrigação e direito
* Artigo 134: Ação e defesa no contrato

**Definição de contrato (Art. 129):**

O contrato é um acordo entre duas ou mais pessoas, que se comprometem a cumprir com certas obrigações. Para ser considerado um contrato, ele deve ter as seguintes características:

* Ser feito por vontade livre e consciente;
* Ter uma clareza e precisão objetiva;
* Estabelecer uma relação de direitos e obrigações entre as partes;
* Ter uma finalidade determinada.

**Tipos de contratos (Art. 130):**

O Código Civil Brasileiro não lista todos os tipos de contratos, mas sim enumera alguns exemplos:

* Contrato de compra e venda
* 

In [28]:
import re
import nltk
import spacy
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import CountVectorizer

# Baixar recursos do NLTK (se necessário)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')

# Carregar o modelo do SpaCy para português
nlp = spacy.load("pt_core_news_sm")

# 1. Limpeza do texto
def limpar_texto(texto):
    texto = re.sub(r'\n+', ' ', texto)  # Remover quebras de linha
    texto = re.sub(r'[^\w\s]', '', texto)  # Remover pontuação
    texto = texto.lower()  # Converter para letras minúsculas
    return texto

# Certifique-se de que 'text' é uma lista e converta para string
documents = load_pdf_documents("codigo_civil.pdf")

if isinstance(text, list):
    text_string = ' '.join(text)
else:
    text_string = text

texto_limpo = limpar_texto(text_string)

# 2. Tokenização
tokens = word_tokenize(texto_limpo, language='portuguese')  # Tokenizar em palavras
sentencas = sent_tokenize(text_string, language='portuguese')  # Tokenizar em sentenças

# 3. Remoção de Stopwords
stop_words = set(stopwords.words('portuguese'))
tokens_sem_stopwords = [palavra for palavra in tokens if palavra not in stop_words]

# 4. Stemming (RSLPStemmer)
stemmer = RSLPStemmer()
tokens_stematizados = [stemmer.stem(palavra) for palavra in tokens_sem_stopwords]

# 5. Lematização com SpaCy
def lematizar_texto(tokens):
    doc = nlp(' '.join(tokens))  # Processar o texto com SpaCy
    return [token.lemma_ for token in doc]

tokens_lematizados = lematizar_texto(tokens_sem_stopwords)

# 6. Bag of Words
vectorizer = CountVectorizer()
# Transformar os tokens em texto novamente, se necessário
texto_para_bow = [' '.join(tokens_lematizados)]

# Criar a matriz BoW
bow_matrix = vectorizer.fit_transform(texto_para_bow)

# Visualizar os resultados
print("\nTexto Original:", text_string)
print("\nTexto Limpo:", texto_limpo)
print("\nTokens:", tokens)
print("\nTokens sem Stopwords:", tokens_sem_stopwords)
print("\nTokens Stematizados:", tokens_stematizados)
print("\nTokens Lematizados:", tokens_lematizados)
print("\nVocabulario BoW:", vectorizer.get_feature_names_out())
print("\nMatriz BoW:")
print(bow_matrix.toarray())


[nltk_data] Downloading package punkt to /home/airex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/airex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/airex/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Carregando: c
Erro ao carregar c: [Errno 2] No such file or directory: 'c'
Carregando: o
Erro ao carregar o: [Errno 2] No such file or directory: 'o'
Carregando: d
Erro ao carregar d: [Errno 2] No such file or directory: 'd'
Carregando: i
Erro ao carregar i: [Errno 2] No such file or directory: 'i'
Carregando: g
Erro ao carregar g: [Errno 2] No such file or directory: 'g'
Carregando: o
Erro ao carregar o: [Errno 2] No such file or directory: 'o'
Carregando: _
Erro ao carregar _: [Errno 2] No such file or directory: '_'
Carregando: c
Erro ao carregar c: [Errno 2] No such file or directory: 'c'
Carregando: i
Erro ao carregar i: [Errno 2] No such file or directory: 'i'
Carregando: v
Erro ao carregar v: [Errno 2] No such file or directory: 'v'
Carregando: i
Erro ao carregar i: [Errno 2] No such file or directory: 'i'
Carregando: l
Erro ao carregar l: [Errno 2] No such file or directory: 'l'
Carregando: .
Erro ao carregar .: [Errno 21] Is a directory: '.'
Carregando: p
Erro ao carregar p: [

ValueError: Nenhum documento foi carregado. Verifique os caminhos dos PDFs.